<a href="https://colab.research.google.com/github/sezinbiner/find_path_genetic_algorithms/blob/main/find_path.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import math
import random

moves=[[-1,0], [-1,-1], [0, -1], [1,-1], [1,0], [1, 1], [0, 1], [-1, 1]]
directions = [1,2,3,4,5,6,7,8];

def find_cost(old_move, new_move):
  if abs(old_move - new_move) == 1 or abs(old_move - new_move) == 7:
    return 1
  elif abs(old_move - new_move) == 2 or abs(old_move - new_move) == 6:
    return 2
  elif abs(old_move - new_move) == 3 or abs(old_move - new_move) == 5:
   return 3
  elif abs(old_move - new_move) == 0:
    return 0
  else:
    return 4


def create_first_route(start, matrix):
  #TODO yönü minimize etme
  route = []
  cost = 0
  route.append(1)
  while len(route) <= 80:
    if start[0] == 0 and start[1] == 0:
      route.append(random.randint(5,7))
    elif start[0] == 8 and start[1] == 8:
      route.append(random.randint(1,3))
    elif start[0] == 8 and start[1] == 0:
      route.append(random.choice([1,7,8]))
    elif start[0] == 0 and start[1] == 8:
      route.append(random.randint(3,5))
    elif start[0] == 8:
      route.append(random.choice([3,2,1,7,8]))
    elif start[0] == 0:
      route.append(random.choice([3,4,5,6,7]))
    elif start[1] == 8:
      route.append(random.choice([1,2,3,4,5]))
    elif start[1] == 0:
      route.append(random.choice([5,6,7,8,1]))
    else:
      route.append(random.randint(1,8))
    
    cost += find_cost(route[-2], route[-1])
    add_move = moves[route[-1]-1]
    
    start[0] += add_move[0]
    start[1] += add_move[1]
    matrix[start[0], start[1]] = 1
  area = len(np.argwhere(matrix == 1))
  return cost, route, start, area

def try_routes(coming_routes, start, size):
  fitness_list = []
  return_routes = []
  count = 0
  for route in coming_routes: 
    cost = 0            #population 5000
    matrix = np.zeros((size,size))
    matrix[start[0], start[1]] = 1
    end = start
    for i in range (len(route)):       #length of one route 9*9-1
      if (i !=0):
        add_move =  moves[route[i]-1]
        end = [end[0] + add_move[0], end[1] + add_move[1]]
        if (end[0] <= 8) and (end[1] <= 8) and (end[0] >= 0) and (end[1] >= 0): #kaç kişi siliniyor bak az silinen varsa create route
          cost += find_cost(route[i-1], route[i])
          matrix[end[0], end[1]] = 1
        else:
          count +=1
    
    print(count)
    area = len(np.argwhere(matrix == 1))
    return_routes.append(route)
    fitness_list.append(find_fitness(start, end, cost, area))
  return return_routes,fitness_list

def calculate_distance(start,end):
    dist = math.sqrt(((start[0]-end[0])**2)+((start[1]-end[1])**2))
    return dist

def find_fitness(start, end, cost, area):
  distance = calculate_distance(start, end)
  fitness = area - cost - distance
  return fitness

def cross_over(routes):
  new_generation = []
  for i in range (len(routes)):
    route1 = random.choice(routes)
    route2 = random.choice(routes)
    new_route1 = route1[0:21] + route2[21:41] + route1[41:61] + route2[61:]
    new_route2 = route2[:21] + route1[21:41] + route2[41:61] + route1[61:]
    new_generation.append(new_route1)
    new_generation.append(new_route2)
    i = i + 1
  return new_generation

def mutation(routes, mutation_ratio, length, population):
  for i in range(int(mutation_ratio)):
    index = random.randint(0,population-1)
    route = routes[index]
    for i in range (3):
      index2 = random.randint(0,length)
      direction = random.randint(1,8)
      route[index2] = direction
    routes[index] = route
  return routes

def main():
  size = 9
  population = 100
  selected = int(population/2)
  generation = 200
  length = size*size-1
  mutation_ratio = 0.01
  matrix = np.zeros((size,size))
  start = [8,0]
  end = start
  matrix[start[0], start[1]] = 1;
  fitness_list = []
  routes = []
  for i in range (population):
    cost, route, end, area = create_first_route(start, matrix)
    routes.append(route)
    fitness_list.append(find_fitness(start, end, cost, area))
    i += 1
  
  zipped = list(zip(routes, fitness_list))
  sorted_routes = sorted(zipped, key = lambda x: x[1])
  unzipped_list = list(zip(*sorted_routes))
  new_generation = unzipped_list[0]

  #//TODO popülasyonu katlama
  for i in range (generation-1):
    cross_generation = cross_over(new_generation)
    mutated_generation = mutation(cross_generation, mutation_ratio*population, length, population)
    new_routes, new_fitness = try_routes(mutated_generation, start, size)
    zipped = list(zip(new_routes, new_fitness))
    sorted_routes = sorted(zipped, key = lambda x: x[1])
    unzipped_list = list(zip(*sorted_routes))
    new_generation = unzipped_list[0]


if __name__ == "__main__":
    main()
  

Streaming output truncated to the last 5000 lines.
after 2
before 1
after 1
before 0
after 0
before 1
after 1
before 2
after 2
before 2
after 2
before 1
after 1
before 0
after 0
before 1
after 1
before 2
after 2
before 3
after 3
before 2
after 2
before 2
after 2
before 2
after 2
before 1
after 1
before 0
after 0
before 0
after 0
before -1
before -2
before -1
before 0
after 0
before 0
after 0
before 0
after 0
before 0
after 0
before 0
after 0
before 0
after 0
before 0
after 0
before -1
before -1
before 0
after 0
before -1
before -2
before -3
before -4
before -3
before -2
before -2
before -3
before -3
before -2
before -3
before -4
before -5
before -4
before -3
before -3
before -2
before -2
before -3
before -4
before -5
before -4
before -4
before -3
before -2
before -3
before -3
before -2
before -2
before -3
before -4
before -3
before -4
before -4
before -3
before -2
before -1
before -1
before -2
before -2
before -3
before -2
before -1
before -2
before -1
before -2
80627
route [1, 1, 6, 4

In [ ]:
new_generation